In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf

In [ ]:
%pip install ultralytics

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

In [ ]:
!mkdir /kaggle/temp/

In [ ]:
orgpaths=[]
tmppaths=[]
for dirname, _, filenames in os.walk('/kaggle/input/indian-licence-plate/licence plate.v1i.darknet/train'):
    for filename in filenames:
        orgpaths+=[os.path.join(dirname, filename)]
        tmppaths+=[os.path.join('/kaggle/temp/', filename[-12:])]
print(len(orgpaths),len(tmppaths))

In [ ]:
for i in range(len(orgpaths)):
    orgpath=orgpaths[i]
    tmppath=tmppaths[i]
    shutil.copy(orgpath,tmppath)

In [ ]:
ipaths=[]
tpaths=[]
iname=[]
tname=[]
for path in tmppaths:
    name=path.split('/')[-1][0:-4]
    if path[-4:]=='.txt' and name!='dbe0b690':
        tpaths+=[path]
        tname+=[name]
    elif path[-4:]=='.jpg' and name!='dbe0b690':
        ipaths+=[path]
        iname+=[name]
ipaths=sorted(ipaths)
tpaths=sorted(tpaths)

print(ipaths[0])
print(len(ipaths))
print(tpaths[0])
print(len(tpaths))

In [ ]:
set(iname)-set(tname)

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(tpaths)):
    tfile=tpaths[i]
    ifile=ipaths[i]
    boxdata+=[np.loadtxt(tfile)]
    boxfile+=[ifile.split('/')[-1]]
print(boxdata[0])
print(boxfile[0])

In [ ]:
BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])

BOX2=BOX.reset_index(drop=True)
display(BOX2)

In [ ]:
class_map = {0:'plate'}

In [ ]:
for i in range(len(BOX2)):
    BOX2.iloc[i,0]=class_map[int(BOX2.iloc[i,0])]
display(BOX2)
display(BOX2.iloc[:,0].value_counts())

In [ ]:
def draw_box(n0):

    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,0]
            x=box.loc[i,1]
            y=box.loc[i,2]
            w=box.loc[i,3]
            h=box.loc[i,4]
            x1=((x-w/2)*W).astype(int)
            y1=((y-h/2)*H).astype(int)
            x2=((x+w/2)*W).astype(int)
            y2=((y+h/2)*H).astype(int)

            if label=='plate':
                cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),5) #green

    return image

In [ ]:
images1=[]
for i in range(len(ipaths)):
    images1+=[draw_box(i)]

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
create_animation(images1)

In [ ]:
os.makedirs('datasets', exist_ok=True)
os.makedirs('datasets/train', exist_ok=True)
os.makedirs('datasets/valid', exist_ok=True)
os.makedirs('datasets/test', exist_ok=True)

In [ ]:
for i in range(len(ipaths)):
    ipath=ipaths[i]
    ifile=ipath.split('/')[-1]
    tpath=tpaths[i]
    tfile=tpath.split('/')[-1]

    if i%3==0:
        copyfile(ipath, f'datasets/train/{ifile}')
        copyfile(tpath, f'datasets/train/{tfile}')
    elif i%3==1:
        copyfile(ipath, f'datasets/valid/{ifile}')
        copyfile(tpath, f'datasets/valid/{tfile}')
    else:
        copyfile(ipath, f'datasets/test/{ifile}')

In [ ]:
import yaml

plate_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =1,
    names =['plate',]
)

with open('plate.yaml', 'w') as outfile:
    yaml.dump(plate_yaml, outfile, default_flow_style=True)

%cat coin.yaml

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=plate.yaml epochs=10 imgsz=640

In [ ]:
tpaths2=[]
for dirname, _, filenames in os.walk('/kaggle/working/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

In [ ]:
best_path0='runs/detect/train/weights/best.pt'
source0='datasets/test'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths[0])

In [ ]:
model2 = YOLO(best_path0)

In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.05 source={source0}

In [ ]:
results = model2.predict(source0,conf=0.05)
print(len(results))

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.boxes).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [ ]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [ ]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,'class']
            x=int(box.loc[i,'x'])
            y=int(box.loc[i,'y'])
            x2=int(box.loc[i,'x2'])
            y2=int(box.loc[i,'y2'])
            #print(label,x,y,x2,y2)

            if label=='plate':
                cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),5) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

In [ ]:
create_animation(images2)